In [21]:
# Importing libraries
import pandas as pd
import numpy as np

# Read csv file into a pandas dataframe
data = pd.read_csv('ardd_fatalities_jan2022.csv',header=None)

# Print the first five rows of data
data.head()

C:\Users\Plath\AppData\Local\Temp\ipykernel_9060\1919386749.py:6: DtypeWarning: Columns (0,2,3,10,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('ardd_fatalities_jan2022.csv',header=None)


,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,Crash ID,State,Month,Year,Dayweek,Time,Crash Type,Bus Involvement,Heavy Rigid Truck Involvement,Articulated Truck Involvement,...,Age,National Remoteness Areas,SA4 Name 2016,National LGA Name 2017,National Road Type,Christmas Period,Easter Period,Age Group,Day of week,Time of day
1,20225010,WA,1,2022,Sunday,2:50,Single,No,No,No,...,20,NaN,NaN,NaN,NaN,No,No,17_to_25,Weekend,Night
2,20224003,SA,1,2022,Friday,21:10,Single,No,No,No,...,21,Remote Australia,South Australia - Outback,Lower Eyre Peninsula (DC),Local Road,No,No,17_to_25,Weekend,Night
3,20223017,Qld,1,2022,Sunday,18:00,Single,No,No,No,...,26,Inner Regional Australia,Ipswich,Somerset (R),Sub-arterial Road,No,No,26_to_39,Weekend,Night
4,20223013,Qld,1,2022,Friday,18:00,Single,No,No,No,...,22,Inner Regional Australia,Wide Bay,Fraser Coast (R),Local Road,No,No,17_to_25,Weekend,Night


In [22]:
data2 = data.copy()

In [23]:

data2.columns = ['Crash_ID', 'State', 'Month', 'Year', 'Dayweek', 'Time', 'Crash_Type', 'Bus_Involvement', 'Heavy_Rigid_Truck_Involvement', 
'Articulated_Truck_Involvement', 'Speed_limit', 'Road_User', 'Gender', 'Age', 'National_Remoteness_Areas', 'SA4_Name_2016', 
                 'National_LGA_Name_2017', 'National_Road_Type', 'Christmas_Period', 'Easter_Period', 'Age_Group', 'Day_Of_Week', 'Time_of_Day']
data2.drop(index=data2.index[0], axis=0, inplace=True) # remove first row of data

In [24]:
print('Number of missing values:')
for col in data2.columns:
    print('\t%s: %d' % (col,data2[col].isna().sum()))

Number of missing values:
	Crash_ID: 0
	State: 0
	Month: 0
	Year: 0
	Dayweek: 0
	Time: 0
	Crash_Type: 0
	Bus_Involvement: 0
	Heavy_Rigid_Truck_Involvement: 0
	Articulated_Truck_Involvement: 0
	Speed_limit: 0
	Road_User: 0
	Gender: 0
	Age: 0
	National_Remoteness_Areas: 45872
	SA4_Name_2016: 45858
	National_LGA_Name_2017: 45854
	National_Road_Type: 45873
	Christmas_Period: 0
	Easter_Period: 0
	Age_Group: 0
	Day_Of_Week: 0
	Time_of_Day: 0


In [27]:
data2 = data2.replace(np.NaN, "?")
for col in data2.columns:
    print('\t%s: %d' % (col,data2[col].isna().sum()))

	Crash_ID: 0
	State: 0
	Month: 0
	Year: 0
	Dayweek: 0
	Time: 0
	Crash_Type: 0
	Bus_Involvement: 0
	Heavy_Rigid_Truck_Involvement: 0
	Articulated_Truck_Involvement: 0
	Speed_limit: 0
	Road_User: 0
	Gender: 0
	Age: 0
	National_Remoteness_Areas: 0
	SA4_Name_2016: 0
	National_LGA_Name_2017: 0
	National_Road_Type: 0
	Christmas_Period: 0
	Easter_Period: 0
	Age_Group: 0
	Day_Of_Week: 0
	Time_of_Day: 0
